In [1]:
import pandas as pd
import numpy as np
import os

import torch
import torchvision



import transformers
import simpletransformers

I0510 22:51:46.227767 139812352890624 file_utils.py:41] PyTorch version 1.5.0 available.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site

In [2]:
from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

In [3]:
dataFolder = '/data1/roshansk/covid_data/'

In [4]:
fileList = os.listdir(dataFolder)

In [5]:
fileList

['messages_cm_mar1_apr23.csv', 'covid19_msgs_2020_02.csv']

In [36]:
df = pd.read_csv(os.path.join(dataFolder, fileList[0]), nrows = 100000)

In [37]:
df.head()

,message_id,user_id,message,created_at_utc,retweeted,retweet_message_id,in_reply_to_message_id,in_reply_to_user_id,favorite_count,retweet_count,...,country_code,tweet_location,tweet_location_short,place_type,coordinates,coordinates_state,fips,county,city,state
0,1238220897720336390,790013818999074820,loving how i predicted the #coronavirus in my ...,2020-03-12 21:50:41,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,12095,Orange County,winter park,FL
1,1238220897871433729,732620465735405569,RT @ShotsRangOut: FNC gave this fool a platfor...,2020-03-12 21:50:41,NaN,1238202374180634624,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,12069,Lake County,Lady Lake,FL
2,1238220898307432448,320608440,The markets is hemorrhaging #trump and his pal...,2020-03-12 21:50:41,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,25005,Bristol County,Freetown,MA
3,1238220898714484736,2186554418,"RT @AdaKirschner: Heads up, y'all.nnThe Nation...",2020-03-12 21:50:41,NaN,1238210905562898435,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,26099,Macomb County,Shelby,MI
4,1238220911972671495,2893026960,RT @ExJWStephen: Jehovah's Witnesses have offi...,2020-03-12 21:50:45,NaN,1238164799713947655,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,39049,Franklin County,Columbus,OH


In [8]:
df.iloc[3]['message']

"RT @AdaKirschner: Heads up, y'all.nnThe National Science Foundation is accepting proposals for non-medical, non-clinical-care research relaâ€¦"

In [10]:
help(BertForSequenceClassification)

Help on class BertForSequenceClassification in module transformers.modeling_bert:

class BertForSequenceClassification(BertPreTrainedModel)
 |  Bert Model transformer with a sequence classification/regression head on top (a linear layer on top of
 |  the pooled output) e.g. for GLUE tasks. 
 |  This model is a PyTorch `torch.nn.Module <https://pytorch.org/docs/stable/nn.html#torch.nn.Module>`_ sub-class.
 |  Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general
 |  usage and behavior.
 |  
 |  Parameters:
 |      config (:class:`~transformers.BertConfig`): Model configuration class with all the parameters of the model.
 |          Initializing with a config file does not load the weights associated with the model, only the configuration.
 |          Check out the :meth:`~transformers.PreTrainedModel.from_pretrained` method to load the model weights.
 |  
 |  Method resolution order:
 |      BertForSequenceClassification
 |      Bert

In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states= True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print('Model loaded')

I0509 21:18:14.544124 140689433306880 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/roshansk/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0509 21:18:14.546376 140689433306880 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2i

Model loaded


In [10]:
sentence1 = "I want to sit on that table"
token1 = tokenizer.encode(sentence1)
decode1 = tokenizer.decode(token1).split(" ")

logits, hidden_states = model(torch.Tensor(token1).unsqueeze(0).long())

sent1 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent1_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [11]:
sentence2 = "The books are placed on the table"
token2 = tokenizer.encode(sentence2)
decode2 = tokenizer.decode(token2).split(" ")
logits, hidden_states = model(torch.Tensor(token2).unsqueeze(0).long())

sent2 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent2_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [12]:
sentence2 = "Lets table the discussion for now and put these books on the table"
token2 = tokenizer.encode(sentence2)
decode2 = tokenizer.decode(token2).split(" ")
logits, hidden_states = model(torch.Tensor(token2).unsqueeze(0).long())

sent2 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent2_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [13]:
from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances

In [21]:
for i in range(len(decode1)):
    word1 = decode1[i]

    
    if word1 != 'table':
        continue
        
    print(f"{word1}")
    
    for j in range(len(decode2)):
        word2 = decode2[j]
        dist = euclidean_distances(sent1_[i].reshape(1,-1), sent2_[j].reshape(1,-1))
        print(f"{word2} -> {dist[0]}")
    print("------")
        

table
[CLS] -> [22.421581]
lets -> [24.649]
table -> [6.0129557]
the -> [21.137703]
discussion -> [20.485327]
for -> [21.555672]
now -> [22.214123]
and -> [20.316694]
put -> [23.219818]
these -> [22.061666]
books -> [22.15198]
on -> [22.081797]
the -> [20.922993]
table -> [6.431495]
[SEP] -> [21.106104]
------


In [15]:
for i in range(len(decode1)):
    word1 = decode1[i]

    
    if word1 != 'table':
        continue
        
    print(f"{word1}")
    
    for j in range(len(decode2)):
        word2 = decode2[j]
        dist = euclidean_distances(sent1_[i].reshape(1,-1), sent2_[j].reshape(1,-1))
        print(f"{word2} -> {dist[0]}")
    print("------")
        

table
[CLS] -> [19.20811]
lets -> [19.866884]
table -> [16.655344]
the -> [18.049162]
discussion -> [14.780564]
for -> [17.403345]
now -> [19.56454]
and -> [17.332333]
put -> [17.118425]
these -> [18.021328]
books -> [15.759896]
on -> [17.438171]
the -> [18.873056]
table -> [11.894768]
[SEP] -> [19.9359]
------


In [53]:
sent1_0

array([[ 0.16855057, -0.28576732, -0.32612562, ..., -0.02757058,
         0.03825327,  0.16399528],
       [-0.35573578,  0.95750636, -1.3448268 , ...,  0.5971607 ,
         0.7391579 ,  0.25808817],
       [-0.98275685,  0.4850043 , -0.74877006, ...,  0.60817134,
         0.9695404 , -0.42965782],
       [-1.1714747 , -1.0952786 , -0.7364164 , ...,  0.50561   ,
        -0.6454608 ,  0.75769067],
       [-0.36429545, -0.16171779,  0.09017371, ..., -0.17849147,
         0.12817715, -0.04511606]], dtype=float32)

In [54]:
sent2_0

array([[ 0.16855057, -0.28576732, -0.32612562, ..., -0.02757058,
         0.03825327,  0.16399528],
       [-0.4164709 ,  0.23321255, -0.65267146, ...,  0.86469525,
        -0.14918818, -0.01466047],
       [-1.7195247 , -0.48560476,  0.39680693, ...,  0.4649588 ,
        -0.8625533 ,  0.70038414],
       [ 0.49153116, -0.76267207,  0.08711345, ...,  0.5580665 ,
        -0.38935202,  0.46139798],
       [-0.5772839 ,  0.773778  , -1.1550461 , ...,  0.49831286,
         0.33618513,  0.45114946],
       [-0.32507333, -0.3187855 , -0.11631773, ..., -0.39602298,
         0.41119915, -0.07755204]], dtype=float32)

In [66]:
scores = []
for i in range(6):
    dist = euclidean_distances(sent1_12[1].reshape(1,-1), sent2_12[i].reshape(1,-1))
    scores.append(dist)
    


In [67]:
dist = euclidean_distances(sent1_0[1].reshape(1,-1), sent2_0[i].reshape(1,-1))

In [69]:
scores

[array([[21.20042]], dtype=float32),
 array([[21.033146]], dtype=float32),
 array([[20.308558]], dtype=float32),
 array([[20.273989]], dtype=float32),
 array([[6.463085]], dtype=float32),
 array([[19.155994]], dtype=float32)]

### Language Modeling

In [16]:
from simpletransformers.language_modeling import LanguageModelingModel

In [18]:
train_args = {
    "output_dir":'CovidBertModel',
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    'fp16':False,
    'num_train_epochs':2
}

model = LanguageModelingModel('bert', 'bert-base-uncased', args=train_args)

I0509 21:35:50.715763 140689433306880 filelock.py:274] Lock 140686028579000 acquired on cache_dir/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
I0509 21:35:50.718173 140689433306880 file_utils.py:479] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /home/roshansk/Covid_Bert/cache_dir/tmp5oly2jph


I0509 21:35:50.858151 140689433306880 file_utils.py:489] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at cache_dir/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0509 21:35:50.859660 140689433306880 file_utils.py:492] creating metadata file for cache_dir/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0509 21:35:50.861044 140689433306880 filelock.py:318] Lock 140686028579000 released on cache_dir/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
I0509 21:35:50.862046 140689433306880 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at cache_dir/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb

I0509 21:35:51.093225 140689433306880 file_utils.py:489] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json in cache at cache_dir/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0509 21:35:51.094940 140689433306880 file_utils.py:492] creating metadata file for cache_dir/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0509 21:35:51.096403 140689433306880 filelock.py:318] Lock 140686031149488 released on cache_dir/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
I0509 21:35:51.097839 140689433306880 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at cache_dir/4dad0251492946e18ac39290fcfe91b89d370fee250ef

I0509 21:35:51.617808 140689433306880 filelock.py:274] Lock 140686031149208 acquired on cache_dir/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock
I0509 21:35:51.619992 140689433306880 file_utils.py:479] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin not found in cache or force_download set to True, downloading to /home/roshansk/Covid_Bert/cache_dir/tmplw21gfh0


I0509 21:36:02.003600 140689433306880 file_utils.py:489] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin in cache at cache_dir/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
I0509 21:36:02.004620 140689433306880 file_utils.py:492] creating metadata file for cache_dir/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
I0509 21:36:02.005798 140689433306880 filelock.py:318] Lock 140686031149208 released on cache_dir/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock
I0509 21:36:02.006551 140689433306880 modeling_utils.py:507] loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at cache_dir/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88f

I0509 21:36:05.976288 140689433306880 modeling_utils.py:601] Weights of BertForMaskedLM not initialized from pretrained model: ['cls.predictions.decoder.bias']
I0509 21:36:05.977300 140689433306880 modeling_utils.py:607] Weights from pretrained model not used in BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']


In [30]:
df.message.iloc[9]

'RT @CoachBMorton: For anybody who wants to know about the Corona virus aka COVID-19 Dr. Drew is here dropping knowledge. nn https://t.co/hVâ€¦'

In [31]:
help(model.tokenizer.get_vocab)

Help on method get_vocab in module transformers.tokenization_bert:

get_vocab() method of transformers.tokenization_bert.BertTokenizer instance
    Returns the vocabulary as a dict of {token: index} pairs. `tokenizer.get_vocab()[token]` is equivalent to `tokenizer.convert_tokens_to_ids(token)` when `token` is in the vocab.



In [32]:
vocab = model.tokenizer.get_vocab()

In [35]:
len(vocab.keys())

30522

In [39]:
trainDf = df[['message','user_id']]
trainDf.columns = ['text','labels']

In [40]:
trainDf.head()

,text,labels
0,loving how i predicted the #coronavirus in my ...,790013818999074820
1,RT @ShotsRangOut: FNC gave this fool a platfor...,732620465735405569
2,The markets is hemorrhaging #trump and his pal...,320608440
3,"RT @AdaKirschner: Heads up, y'all.nnThe Nation...",2186554418
4,RT @ExJWStephen: Jehovah's Witnesses have offi...,2893026960


In [42]:
trainDf.to_csv('CovidTrainDf.csv',index=False, index_label=False)

In [43]:
model.train_model('CovidTrainDf.csv')

I0509 21:45:28.468431 140689433306880 language_modeling_utils.py:177]  Creating features from dataset file at cache_dir/


I0509 21:45:36.366803 140689433306880 language_modeling_utils.py:225]  Saving features into cached file cache_dir/bert_cached_lm_126_CovidTrainDf.csv
I0509 21:45:36.503617 140689433306880 language_modeling_model.py:423]  Training started


Running loss: 5.043767

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Running loss: 4.698415

...

I0509 22:24:28.330224 140689433306880 configuration_utils.py:142] Configuration saved in CovidBertModel/checkpoint-8000/config.json
I0509 22:24:28.753625 140689433306880 modeling_utils.py:343] Model weights saved in CovidBertModel/checkpoint-8000/pytorch_model.bin


Running loss: 1.654968

I0509 22:28:57.287918 140689433306880 configuration_utils.py:142] Configuration saved in CovidBertModel/checkpoint-8918-epoch-2/config.json


I0509 22:28:57.713877 140689433306880 modeling_utils.py:343] Model weights saved in CovidBertModel/checkpoint-8918-epoch-2/pytorch_model.bin
I0509 22:28:58.506895 140689433306880 configuration_utils.py:142] Configuration saved in CovidBertModel/config.json


I0509 22:28:58.838877 140689433306880 modeling_utils.py:343] Model weights saved in CovidBertModel/pytorch_model.bin
I0509 22:28:58.859014 140689433306880 language_modeling_model.py:352]  Training of bert model complete. Saved to CovidBertModel.


In [45]:
len(model.tokenizer.get_vocab())

30522

In [69]:
model = BertForSequenceClassification.from_pretrained('CovidBertModel', output_hidden_states= True)

I0510 23:31:13.905913 139812352890624 configuration_utils.py:281] loading configuration file CovidBertModel/config.json
I0510 23:31:13.908410 139812352890624 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "bert",
  "no_repeat_ngram_size": 0,
  "num_attention

In [75]:
tokenizer = BertTokenizer.from_pretrained('CovidBertModel/')

I0510 23:35:14.406204 139812352890624 tokenization_utils.py:420] Model name 'CovidBertModel/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'CovidBertModel/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0510 23:35:14.408357 139812352890624 tokenization_utils.py:449] Didn't find file CovidBertModel/added_tokens.json. We won't load it.
I0510 23:35:14.409713 139812352890624 tokenization_utils.py:502] loading file CovidBer

In [76]:
sentence1 = "Covid Corona virus covid-19"

In [77]:
token1 = tokenizer.encode(sentence1)
decode1 = tokenizer.decode(token1).split(" ")

In [71]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir = './BertTokenizer')

I0510 23:34:43.597817 139812352890624 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./BertTokenizer/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [68]:
tokenizer.vocab

OrderedDict([('[PAD]', 0),
             ('[unused0]', 1),
             ('[unused1]', 2),
             ('[unused2]', 3),
             ('[unused3]', 4),
             ('[unused4]', 5),
             ('[unused5]', 6),
             ('[unused6]', 7),
             ('[unused7]', 8),
             ('[unused8]', 9),
             ('[unused9]', 10),
             ('[unused10]', 11),
             ('[unused11]', 12),
             ('[unused12]', 13),
             ('[unused13]', 14),
             ('[unused14]', 15),
             ('[unused15]', 16),
             ('[unused16]', 17),
             ('[unused17]', 18),
             ('[unused18]', 19),
             ('[unused19]', 20),
             ('[unused20]', 21),
             ('[unused21]', 22),
             ('[unused22]', 23),
             ('[unused23]', 24),
             ('[unused24]', 25),
             ('[unused25]', 26),
             ('[unused26]', 27),
             ('[unused27]', 28),
             ('[unused28]', 29),
             ('[unused29]', 30),
  

In [78]:
token1

[101, 30522, 21887, 7865, 30522, 1011, 2539, 102]

In [79]:
decode1

['[CLS]', 'covid', 'corona', 'virus', 'covid', '-', '19', '[SEP]']

In [56]:
tokenizer.ids_to_tokens[30522]

KeyError: 30522

In [48]:
model.config.vocab_size

30522

In [42]:
tokenizer.vocab_size

30522

In [35]:
help(model.resize_token_embeddings)

Help on method resize_token_embeddings in module transformers.modeling_utils:

resize_token_embeddings(new_num_tokens=None) method of transformers.modeling_bert.BertForSequenceClassification instance
    Resize input token embeddings matrix of the model if new_num_tokens != config.vocab_size.
    Take care of tying weights embeddings afterwards if the model class has a `tie_weights()` method.
    
    Arguments:
    
        new_num_tokens: (`optional`) int:
            New number of tokens in the embedding matrix. Increasing the size will add newly initialized vectors at the end. Reducing the size will remove vectors from the end.
            If not provided or None: does nothing and just returns a pointer to the input tokens ``torch.nn.Embeddings`` Module of the model.
    
    Return: ``torch.nn.Embeddings``
        Pointer to the input tokens Embeddings Module of the model



In [22]:
BertTokenizer.vocab_files_names

{'vocab_file': 'vocab.txt'}

In [62]:
tokenizer.add_tokens(['covid-19'])

I0510 23:25:41.918249 139812352890624 tokenization_utils.py:663] Adding covid-19 to the vocabulary


1

In [63]:
len(tokenizer)

30524

In [65]:
tokenizer.encode_plus('Here go covid-19 the time')

{'input_ids': [101, 2182, 2175, 30522, 1011, 2539, 1996, 2051, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [66]:
help(tokenizer.from_pretrained)

Help on method from_pretrained in module transformers.tokenization_utils:

from_pretrained(*inputs, **kwargs) method of builtins.type instance
    Instantiate a :class:`~transformers.PreTrainedTokenizer` (or a derived class) from a predefined tokenizer.
    
    Args:
        pretrained_model_name_or_path: either:
    
            - a string with the `shortcut name` of a predefined tokenizer to load from cache or download, e.g.: ``bert-base-uncased``.
            - a string with the `identifier name` of a predefined tokenizer that was user-uploaded to our S3, e.g.: ``dbmdz/bert-base-german-cased``.
            - a path to a `directory` containing vocabulary files required by the tokenizer, for instance saved using the :func:`~transformers.PreTrainedTokenizer.save_pretrained` method, e.g.: ``./my_model_directory/``.
            - (not applicable to all derived classes, deprecated) a path or url to a single saved vocabulary file if and only if the tokenizer only requires a single vocabul